In [1]:
!pip install importlib-metadata==4.13.0
!pip in!pip install importlib-metadata==4.13.0stall importlib-metadata==4.13.0
import os
import functools
import time
!pip install omegaconf
from omegaconf import OmegaConf

import gym
!pip install git+https://github.com/osigaud/bbrl_gym.git
!pip install git+https://github.com/osigaud/bbrl.git



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0
ERROR: unknown command "in!pip"
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=d22f09a2541bd287221339c04a7941e4614c8eccd5e0652697fef778df0a97a1
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/osigaud/bbrl_gym.git to /tmp/pip-req-build-rrffejis
  Running command git clone --filter=blob:none --quiet https://github.com/osigaud/bbrl_gym.git /tmp/pip-req-build-rrffejis
  Resolved https://github.com/osigaud/bbrl_gym.git to commit 5557075ecd7d4171ac0c21be3c69a94bcae655a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 KB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.6 MB/s eta 0:00:00
  Created wheel for bbrl-gym: filename=bbrl_gym-1.2.5-py3-none-any.whl size=17599 sha256=0cd769c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/osigaud/bbrl.git to /tmp/pip-req-build-s3ld0m6q
  Running command git clone --filter=blob:none --quiet https://github.com/osigaud/bbrl.git /tmp/pip-req-build-s3ld0m6q
  Resolved https://github.com/osigaud/bbrl.git to commit 5e8530b3eba8b14084deafe83795b93afca618f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bbrl: filename=bbrl-0.1.9-py3-none-any.whl size=50513 sha256=689555ece9e928e8c370289f822c5082bcc82fae03f413db3af213f00ae79e8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-1n7m9y4c/wheels/cc/1d/b5/74c1c5cbfde8f8270cbcfa773985a95000d73eb26c771c7eff
Successfully built bbrl


In [2]:
import bbrl
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class

# The workspace is the main class in BBRL, this is where all data is collected and stored
from bbrl.workspace import Workspace

# Agents(agent1,agent2,agent3,...) executes the different agents the one after the other
# TemporalAgent(agent) executes an agent over multiple timesteps in the workspace, 
# or until a given condition is reached
from bbrl.agents import Agents, RemoteAgent, TemporalAgent

#Replay Buffer
from bbrl.utils.replay_buffer import ReplayBuffer

# AutoResetGymAgent is an agent able to execute a batch of gym environments
# with auto-resetting. These agents produce multiple variables in the workspace: 
# ’env/env_obs’, ’env/reward’, ’env/timestep’, ’env/done’, ’env/initial_state’, ’env/cumulated_reward’, 
# ... When called at timestep t=0, then the environments are automatically reset. 
# At timestep t>0, these agents will read the ’action’ variable in the workspace at time t − 1
from bbrl.agents.gymb import AutoResetGymAgent, NoAutoResetGymAgent
# Not present in the A2C version...
from bbrl.utils.logger import TFLogger

In [4]:
def build_mlp(sizes, activation, output_activation=nn.Identity()):
    print(sizes)
    layers = []
    for j in range(len(sizes) - 1):
        act = activation if j < len(sizes) - 2 else output_activation
        layers = [nn.Linear(sizes[j], sizes[j + 1]), act]
    return nn.Sequential(*layers)

In [5]:
class DiscreteQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.model = build_mlp(
            [state_dim] + list(hidden_layers) + [action_dim], activation=nn.ReLU()
        )

    def forward(self, t, choose_action=True, **kwargs):
        obs = self.get(("env/env_obs", t))
        #print(obs)
        q_values = self.model(obs).squeeze(-1)
        self.set(("q_values", t), q_values)
        if choose_action:
            action = q_values.argmax(1)
            self.set(("action", t), action) 

In [46]:
class LSTMQAgent(Agent):
    def __init__(self, state_dim, hidden_dim, nb_layers, action_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(state_dim, hidden_dim, nb_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, action_dim)
        self.nb_layers = nb_layers

    def forward(self, t, choose_action=True, **kwargs):
        obs = self.get(("env/env_obs", t))
        # print(obs)
        batch_size = obs.shape[0]
        # L'état caché initial h0 est utilisé pour initialiser la sortie de l'unité LSTM pour la première séquence d'entrée, 
        # et la sortie de la première séquence devient l'état caché pour la deuxième séquence d'entrée, et ainsi de suite.
        # La mémoire cellulaire c0 est également utilisée pour stocker des informations importantes sur les séquences précédentes et est passée d'une séquence à l'autre.
        h0 = torch.zeros(self.nb_layers, batch_size, self.hidden_dim).to(obs.device)
        c0 = torch.zeros(self.nb_layers, batch_size, self.hidden_dim).to(obs.device)
        # print(obs.shape, h0.shape,c0.shape)

        lstm_out, _ = self.lstm(obs.unsqueeze(0), (h0, c0))  
        # lstm_out : batch_size x sequence_length x state_dim
        q_values = self.fc(lstm_out[:, -1, :])
        self.set(("q_values", t), q_values)
        if choose_action:
            action = q_values.argmax(1)
            self.set(("action", t), action)


In [36]:
def make_env(env_name):
    return gym.make(env_name)

In [37]:
class EGreedyActionSelector(Agent):
    def __init__(self, epsilon):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, t, **kwargs):
        q_values = self.get(("q_values", t))
        nb_actions = q_values.size()[1]
        size = q_values.size()[0]
        is_random = torch.rand(size).lt(self.epsilon).float()
        random_action = torch.randint(low=0, high=nb_actions, size=(size,))
        max_action = q_values.max(1)[1]
        action = is_random * random_action + (1 - is_random) * max_action
        action = action.long()
        self.set(("action", t), action)

In [38]:
def get_env_agents(cfg):
    train_env_agent = AutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.n_envs,
        cfg.algorithm.seed,
    )
    eval_env_agent = NoAutoResetGymAgent(
    get_class(cfg.gym_env),
    get_arguments(cfg.gym_env),
    cfg.algorithm.nb_evals,
    cfg.algorithm.seed,
    )
    return train_env_agent, eval_env_agent

In [39]:
def create_dqn_agent(cfg, train_env_agent, eval_env_agent):
    obs_size, act_size = train_env_agent.get_obs_and_actions_sizes()
    #critic = DiscreteQAgent(obs_size, cfg.algorithm.architecture.hidden_size, act_size)
    critic = LSTMQAgent(obs_size, cfg.algorithm.architecture.hidden_size, 2, act_size)
    explorer = EGreedyActionSelector(cfg.algorithm.epsilon)
    q_agent = TemporalAgent(critic)
    tr_agent = Agents(train_env_agent, critic, explorer)
    ev_agent = Agents(eval_env_agent, critic)

    # Get an agent that is executed on a complete workspace
    train_agent = TemporalAgent(tr_agent)
    eval_agent = TemporalAgent(ev_agent)
    train_agent.seed(cfg.algorithm.seed)
    return train_agent, eval_agent, q_agent


In [40]:
class Logger():

  def __init__(self, cfg):
    self.logger = instantiate_class(cfg.logger)

  def add_log(self, log_string, loss, epoch):
    self.logger.add_scalar(log_string, loss.item(), epoch)

  # Log losses
  def log_losses(self, cfg, epoch, critic_loss, entropy_loss, a2c_loss):
    self.add_log("critic_loss", critic_loss, epoch)
    self.add_log("entropy_loss", entropy_loss, epoch)
    self.add_log("actor_loss", a2c_loss, epoch)


In [41]:
# Configure the optimizer over the q agent
def setup_optimizers(cfg, q_agent):
    optimizer_args = get_arguments(cfg.optimizer)
    parameters = q_agent.parameters()
    optimizer = get_class(cfg.optimizer)(parameters, **optimizer_args)
    return optimizer

In [42]:
def compute_critic_loss(cfg, reward, must_bootstrap, q_values, action):
    """_summary_

    Args:
        cfg (_type_): _description_
        reward (torch.Tensor): A (T x B) tensor containing the rewards
        must_bootstrap (torch.Tensor): a (T x B) tensor containing 0 if the episode is completed at time $t$
        q_values (torch.Tensor): a (T x B x A) tensor containing Q values
        action (torch.LongTensor): a (T) long tensor containing the chosen action

    Returns:
        torch.Scalar: The DQN loss and the temporal difference
    """
    # We compute the max of Q-values over all actions
    max_q = q_values.max(-1)[0].detach()

    # To get the max of Q(s_{t+1}, a), we take max_q[1:]
    # The same about must_bootstrap. 
    target = (
        reward[:-1] + cfg.algorithm.discount_factor * max_q * must_bootstrap.int()
    )
    # To get Q(s,a), we use torch.gather along the 3rd dimension (the action)
    act = action[0].unsqueeze(-1)
    qvals = torch.gather(q_values[0], dim=1, index=act).squeeze()

    # Compute the temporal difference (use must_boostrap as to mask out finished episodes)
    td = target - qvals 
    # Compute critic loss
    td_error = td**2
    critic_loss = td_error.mean()
    return critic_loss, td
    

In [48]:
def run_dqn(cfg, max_grad_norm=0.5):
    # 1)  Build the  logger
    logger = Logger(cfg)
    best_reward = -10e9

    # 2) Create the environment agent
    train_env_agent, eval_env_agent = get_env_agents(cfg)

    # 3) Create the DQN-like Agent
    train_agent, eval_agent, q_agent = create_dqn_agent(
        cfg, train_env_agent, eval_env_agent
    )

    # 5) Configure the workspace to the right dimension
    # Note that no parameter is needed to create the workspace.
    # In the training loop, calling the agent() and critic_agent()
    # will take the workspace as parameter
    train_workspace = Workspace()  # Used for training

    # 6) Configure the optimizer over the a2c agent
    optimizer = setup_optimizers(cfg, q_agent)
    nb_steps = 0
    tmp_steps = 0

    # 7) Training loop
    for epoch in range(cfg.algorithm.max_epochs):
        # print(epoch)
        # Execute the agent in the workspace
        if epoch > 0:
            train_workspace.zero_grad()
            train_workspace.copy_n_last_steps(1)
            train_agent(
                train_workspace, t=1, n_steps=cfg.algorithm.n_steps - 1, stochastic=True
            )
        else:
            train_agent(
                train_workspace, t=0, n_steps=cfg.algorithm.n_steps, stochastic=True
            )

        transition_workspace = train_workspace.get_transitions()

        # The q agent needs to be executed on the rb_workspace workspace (gradients are removed in workspace).
        
      
        q_values, done, truncated, reward, action = transition_workspace[
            "q_values", "env/done", "env/truncated", "env/reward", "action"
        ]

        nb_steps += len(action[0]) * cfg.algorithm.n_envs

        # Determines whether values of the critic should be propagated
        # True if the episode reached a time limit or if the task was not done
        # See https://colab.research.google.com/drive/1erLbRKvdkdDy0Zn1X_JhC01s1QAt4BBj?usp=sharing
        must_bootstrap = torch.logical_or(~done[1], truncated[1])

        # Compute critic loss
        critic_loss, td = compute_critic_loss(
            cfg, reward, must_bootstrap, q_values, action
        )

        # Store the loss for tensorboard display
        logger.add_log("critic_loss", critic_loss, nb_steps)

        optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(q_agent.parameters(), max_grad_norm)
        obs = train_workspace["env/env_obs"].shape
        # print(obs)
        
        optimizer.step()

        if nb_steps - tmp_steps > cfg.algorithm.eval_interval:
            tmp_steps = nb_steps

            eval_workspace = Workspace()  # Used for evaluation

            eval_agent(
                eval_workspace, t=0, stop_variable="env/done", choose_action=True
            )

            rewards = eval_workspace["env/cumulated_reward"][-1]

            mean = rewards.mean()
            logger.add_log("reward", mean, nb_steps)
            print(f"epoch: {epoch}, reward: {mean}")
            if cfg.save_best and mean > best_reward:
                best_reward = mean
                directory = "./dqn0_critic/"
                if not os.path.exists(directory):
                    os.makedirs(directory)
                filename = directory + "dqn0_" + str(mean.item()) + ".agt"
                eval_agent.save_model(filename)


In [44]:
params={
  "save_best": False,
  "logger":{
    "classname": "bbrl.utils.logger.TFLogger",
    "log_dir": "./tmp/" + str(time.time()),
    "cache_size": 10000,
    "every_n_seconds": 10,
    "verbose": False,    
    },

  "algorithm":{
    "seed": 5,
    "epsilon": 0.02,
    "n_envs": 1,
    "n_steps": 100,
    "eval_interval": 2000,
    "nb_evals": 1,
    "gae": 0.8,
    "max_epochs": 3500,
    "discount_factor": 0.99,
    "batch_size": 256,
    "max_buffer_size": 1e5,
    "target_critic_update": 5000,
    #"architecture":{"hidden_size": [256,256]},
    "architecture":{"hidden_size": 256, "nb_layers":2},
  },
  "gym_env":{
    "classname": "__main__.make_env",
    "env_name": "CartPole-v1",
  },
  "optimizer":        # print(batch_size)

  {
    "classname": "torch.optim.Adam",
    "lr": 2e-3,
  }
}

In [49]:
config=OmegaConf.create(params)
torch.manual_seed(config.algorithm.seed)
run_dqn(config)

epoch: 22, reward: 9.0
epoch: 45, reward: 10.0
epoch: 68, reward: 9.0
epoch: 91, reward: 10.0
epoch: 114, reward: 10.0
epoch: 137, reward: 10.0
epoch: 160, reward: 9.0


KeyboardInterrupt: ignored

L'environnement CartPole-v1 est un environnement de simulation où l'objectif est de maintenir un poteau en équilibre en déplaçant un chariot de gauche à droite. L'environnement retourne deux types d'informations : l'observation et l'action.

La variable env.observation_space correspond à l'espace des observations de l'environnement. Dans ce cas, c'est un espace continu de 4 dimensions (Box) avec les bornes suivantes : [-4.8, 4.8] pour la position du chariot, [-inf, inf] pour la vitesse du chariot, [-0.42, 0.42] pour l'angle du poteau et [-inf, inf] pour la vitesse angulaire du poteau.

La variable env.action_space correspond à l'espace des actions possibles que l'on peut prendre dans l'environnement. Dans ce cas, c'est un espace discret de 2 actions possibles (Discrete) : 0 et 1, qui correspondent à déplacer le chariot à gauche ou à droite, respectivement.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./tmp